!jt -t chesterish
# !jt -r

In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
from tqdm import tqdm
from itertools import product #combinations_with_replacement
import binascii
%matplotlib notebook

import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation

import copy
from collections import Counter
import pandas as pd

In [2]:
## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 8
nr_individuals = 500

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
ws = 40
world_size_x = ws
world_size_y = ws

##TODo if world_size_x != world_size_y fit then limit coord_full_list 

coord_full_list = np.array(list(product(range(ws), repeat=2)))
ind = np.random.choice(len(coord_full_list), nr_individuals, replace=False)
pos = coord_full_list[ind]

## initial brain and position generator

result = calculate_individual_output_weights(individuals)

## add genome

for indiv in result:
    result[indiv]['genome'] = [i.hex_id for i in individuals[indiv]]
    
## assign position and remove brains without output

indiv_to_del = []
for indiv in result:
    if result[indiv]['out']:
        result[indiv]['position'] = [list(pos[indiv])]
    else:
        indiv_to_del.append(indiv)

for key in indiv_to_del:
    del result[key] 

In [ ]:
k = ws
n = 0
pbar = tqdm(total=k, initial=n)

while k>n: 
    pos_list = [tuple(result[obj]['position'][-1]) for obj in result]
    res = list(set([ele for ele in pos_list if pos_list.count(ele) > 1]))
    pbar.update(1)
    for indiv in result:
        x, y = result[indiv]['position'][-1][0], result[indiv]['position'][-1][1]
        if n<1:
            calculate_position(result, indiv, x, y, world_size_x, world_size_y)    
        elif n >= 1:
            apply_input(result, indiv)
            calculate_position(result, indiv, x, y, world_size_x, world_size_y)
    last_pos_list = {obj:result[obj]['position'][-1] for obj in result}
    prevent_overlap_movement(last_pos_list, result)
    n += 1
pbar.close()

## overlap check
df = pd.DataFrame()
for key in result:
    df[key] = pd.Series(result[key]['position']).apply(lambda x: '.'.join((str(x[0]),str(x[1]))))

print(Counter(df.iloc[-1,:]).most_common(10))

## visualisation
coords = generate_dictionary_of_coords(result)
 
fig, ax = plt.subplots(figsize=(6,6))
anime = FuncAnimation(
    fig = fig,
    func = update,
    frames = len(coords),
    interval = 10, repeat=False
)


 35%|████████████████████████████▋                                                     | 14/40 [00:25<01:04,  2.49s/it]

### mutation

hexa to binary

In [ ]:
result[0]['genome']

In [ ]:
aa = ['d5bba35f',
 'd2b747d2',
 '6892268f',
 'd614a25a',
 '5a4dddcf',
 'ca99c460',
 '7cddec5a',
 'e7bc8cc5']
print(aa)
genome = copy.copy(aa)

for gene_nr, gene in enumerate(genome):
    binary_gene = hexval_to_bin(gene)
    binary_mutated = mutation(binary_gene, weight=0.09)
    if binary_mutated != binary_gene:
        genome[gene_nr] = hex(int(binary_mutated, 2))[2:]
